In [1]:
import cv2
import os
import dlib

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

from sklearn.metrics import f1_score

In [2]:
transform = transforms.Compose([
    transforms.Resize((256, 256)),
    transforms.ToTensor(),
])

In [3]:
traindataset = datasets.ImageFolder(root="images/", transform=transform)
testdataset = datasets.ImageFolder(root="test/", transform=transform)

train_dataloader = DataLoader(traindataset, batch_size=8, shuffle=True)
test_dataloader = DataLoader(testdataset, batch_size=8)

In [4]:
# Verileri incele
for images, labels in train_dataloader:
    print(f"Images batch shape: {images.size()}")  # Her batch'in boyutunu göster
    print(f"Labels: {labels}")  # Her batch'in etiketlerini göster (0 = kubilay, 1 = furkan)
    break  # İlk batch'i gösterip döngüden çık

Images batch shape: torch.Size([8, 3, 256, 256])
Labels: tensor([1, 1, 1, 0, 0, 0, 0, 0])


In [5]:
# train_dataloader'dan bir batch alın
images, labels = next(iter(train_dataloader))

# İlk görüntünün boyutunu yazdırın
print(f"İlk görüntünün boyutu: {images[7].shape}")

İlk görüntünün boyutu: torch.Size([3, 256, 256])


In [6]:
# Sınıfları görüntüle
print(traindataset.classes)  # ['furkan', 'kubilay']
print(traindataset.class_to_idx)  # {'furkan': 0, 'kubilay': 1}

['furkan', 'kubilay']
{'furkan': 0, 'kubilay': 1}


In [7]:
# CNN tabanlı FaceNet benzeri model
faceNetModel = nn.Sequential(
    # 1. Conv Bloğu
    nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1),  # Giriş: 3, Çıkış: 64
    nn.BatchNorm2d(64),  # Burada çıkış kanalları 64 olmalı
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),  # Boyutları yarıya indirir
    
    # 2. Conv Bloğu
    nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1),  # Giriş: 64, Çıkış: 128
    nn.BatchNorm2d(128),  # Burada çıkış kanalları 128 olmalı
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),  # Boyutları yarıya indirir

    # 3. Conv Bloğu
    nn.Conv2d(128, 256, kernel_size=3, stride=1, padding=1),  # Giriş: 128, Çıkış: 256
    nn.BatchNorm2d(256),  # Burada çıkış kanalları 256 olmalı
    nn.ReLU(),
    nn.MaxPool2d(kernel_size=2, stride=2),  # Boyutları yarıya indirir

    # Flatten ve Fully Connected katmanlar
    nn.Flatten(),
    nn.Linear(256 * 32 * 32, 256),  # Girdi boyutu: 256 x 32 x 32, toplam 256 * 32 * 32 özellik
    nn.ReLU(),
    nn.Dropout(0.5),  # Overfitting'i önlemek için dropout
    
    nn.Linear(256, 128),  # 128 boyutlu embedding
    nn.ReLU(),
    nn.Linear(128, 2)  # 2 sınıf: kubilay ve furkan
)

In [8]:
# Kayıp fonksiyonu ve optimizasyon
criterion = nn.CrossEntropyLoss()  # Çok sınıflı sınıflandırma kayıp fonksiyonu
optimizer = optim.Adam(faceNetModel.parameters(), lr=0.001)  # Adam optimizasyon algoritması

device = torch.device("cuda" if torch.cuda.is_available else "cpu")
faceNetModel = faceNetModel.to(device)

In [10]:
# Eğitim fonksiyonu
def train(model, dataloader, criterion, optimizer, epochs=10):
    model.train()  # Modeli eğitim moduna al

    for epoch in range(epochs):
        running_loss = 0.0

        for images, labels in dataloader:
            # Verileri GPU'ya taşı
            images, labels = images.to(device), labels.to(device)

            # Sıfırla gradientler
            optimizer.zero_grad()

            # İleri geçiş
            outputs = model(images)

            # Kayıp hesapla
            loss = criterion(outputs, labels)

            # Geri geçiş ve optimizasyon
            loss.backward()
            optimizer.step()

            # Kayıp değerini biriktir
            running_loss += loss.item()

        # Her epoch sonunda kayıp değerini göster
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {running_loss/len(dataloader):.4f}")

In [11]:
train(faceNetModel, train_dataloader, criterion, optimizer, 10)

Epoch [1/10], Loss: 2.9733
Epoch [2/10], Loss: 0.8008
Epoch [3/10], Loss: 0.0000
Epoch [4/10], Loss: 0.0000
Epoch [5/10], Loss: 0.0000
Epoch [6/10], Loss: 0.2179
Epoch [7/10], Loss: 0.8957
Epoch [8/10], Loss: 0.1373
Epoch [9/10], Loss: 3.6393
Epoch [10/10], Loss: 0.1698


In [12]:
torch.save(faceNetModel, "FirstTrainFaceNet.pth")

In [13]:
loadedModel = torch.load("FirstTrainFaceNet.pth")

C:\Users\celik\AppData\Local\Temp\ipykernel_9708\2002457176.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loadedModel = torch.load("FirstTrainFaceNet.pth")


In [14]:
loadedModel.to(device)

Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU()
  (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (4): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (5): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (6): ReLU()
  (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (8): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (9): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (10): ReLU()
  (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (12): Flatten(start_dim=1, end_dim=-1)
  (13): Linear(in_features=262144, out_features=256, bias=True)
  (14): ReLU()
  (15): Dropout(p=0.5, inplace=False)
  (16): Linear(in_features=256, out_features=128, bias=T

In [15]:
def test(model, dataloader):
    model.eval()  # Modeli değerlendirme moduna al
    correct = 0
    total = 0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():  # Gradyan hesaplamayı kapat
        for images, labels in dataloader:
            images, labels = images.to(device), labels.to(device)  # Verileri GPU'ya taşı
            
            # Modelin tahminlerini al
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            
            # Doğru tahminleri say
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            
            # Tahmin ve etiketleri listeye ekle
            all_preds.extend(predicted.cpu().numpy())  # Tahminleri CPU'ya taşı ve listeye ekle
            all_labels.extend(labels.cpu().numpy())  # Gerçek etiketleri CPU'ya taşı ve listeye ekle

    # Doğruluk oranını hesapla
    accuracy = 100 * correct / total
    print(f"Test Accuracy: {accuracy:.2f}%")

    # F1 skorunu hesapla (weighted average)
    f1 = f1_score(all_labels, all_preds, average='weighted')
    print(f"F1 Score: {f1:.4f}")

In [16]:
test(loadedModel, test_dataloader)

Test Accuracy: 27.27%
F1 Score: 0.2143
